## First test

In [23]:
from selenium import webdriver
import time

driver = webdriver.Firefox()
driver.get("https://www.uptodate.com/contents/pathophysiology-and-natural-history-of-mitral-stenosis",
           )

In [1]:
from big_seo.crawler.selenium_proxy import SeleniumProxy

In [7]:
proxy = SeleniumProxy()

In [ ]:
a = proxy.get(url='https://www.uptodate.com/contents/pathophysiology-and-natural-history-of-mitral-stenosis', params={},stream=False)

In [27]:
from selenium.webdriver.common.by import By

In [ ]:
driver.find_element(value='body',by=By.TAG_NAME).get_attribute("outerHTML")

In [10]:
with open('test.html','w') as f:
    f.write(driver.find_element(value='topicText').get_attribute("outerHTML"))

In [1]:
from big_seo.common.lang import DocumentLang
DocumentLang.VI, str(DocumentLang.VI)

(<DocumentLang.VI: ('vi', {'vi': 'Tiếng Việt', 'en': 'Vietnamese'})>, 'vi')

In [5]:
f"""Bạn là một chuyên gia dịch thuật trong lĩnh vực {{realm}}, bạn có khả dịch chính xác những bài viết bằng {DocumentLang.EN.get_full_lang_name(DocumentLang.VI)} sang {DocumentLang.VI.get_full_lang_name(DocumentLang.VI)}.
            Dịch bài sau sang {DocumentLang.VI.get_full_lang_name(DocumentLang.VI)}: {{main_text}}"""

'Bạn là một chuyên gia dịch thuật trong lĩnh vực {realm}, bạn có khả dịch chính xác những bài viết bằng Tiếng Anh sang Tiếng Việt.\n            Dịch bài sau sang Tiếng Việt: {main_text}'

In [6]:
with open('test_u2d_2.html','r') as f:
    page_content = f.read()

In [7]:
from bs4 import BeautifulSoup
import requests
import json

In [8]:
soup = BeautifulSoup(page_content,'html.parser')

In [9]:
from big_seo.llm.implement.parser import UpToDateWebPageParser
from big_seo.llm.implement.indexer import WebPageDocument
from big_seo.crawler.core import WebPage
parser = UpToDateWebPageParser()

In [10]:
webpage = WebPage(b'test',
                  'test',
                  'uptodate.com',
                  'mistral stenosis',
                  None,
                  page_content,
                  headings={},
                  main_content='',
                  keywords=[],
                  rank=0)
doc = WebPageDocument(webpage)

In [11]:
parser.feed(doc)

In [12]:
a = parser.parse(in_doc=True,concat_to='h2')

In [ ]:
a

## Test indexer

In [1]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
import uuid
from functools import partial
import datetime

In [2]:
start = datetime.datetime.now()

In [3]:
from big_seo.crawler.api import Crawler, UpToDateCrawler
from big_seo.llm.implement.indexer import WebPageIndexer, InPageIndexer, UpToDateInPageIndexer
from big_seo.llm.implement.parser import WebPageParser, TranslationParser, UpToDateWebPageParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from big_seo.llm.implement.embedding_model import LLamaIndexEmbedding
from big_seo.llm.implement.parser import OutlineDocumentCreator

from big_seo.llm.implement.data_model import WebPageDocument, OutlineDocumentDBType, Base
from big_seo.common.lang import DocumentLang
hug_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
embed_model = LLamaIndexEmbedding(hug_model)
crawler = UpToDateCrawler()

In [4]:
outline_doc_creator = OutlineDocumentCreator()
parser = UpToDateWebPageParser(doc_creator=outline_doc_creator)
parser.parse = partial(parser.parse,nested=False, in_doc=True,concat_to='h2')
# parser_h2 = UpToDateWebPageParser(doc_creator=outline_doc_creator)


In [5]:
from big_seo.translator.core import GeminiHealthcareTranslator
translator = GeminiHealthcareTranslator(model='gemini-pro')
translator_parser = TranslationParser(parser,translator,
                               input_lang=DocumentLang.EN,
                               target_lang=DocumentLang.VI)


In [6]:
main_topic = 'mistral stenosis'
translator.translate = partial(translator.translate,other_instruction=' liên quan đến hẹp van hai lá')
webpages = crawler.search(main_topic,n=2)


In [7]:
from big_seo.storage.qdrant import QdrantDB

vec_db = QdrantDB(host='localhost',
                port=6333)
Base.init_instance(vec_db=vec_db)

In [8]:
uptodate_indexer = UpToDateInPageIndexer(embedding_model=embed_model,
              parser=translator_parser,
              Base_model=Base)

for page in webpages:
    page_doc = WebPageDocument(page)
    uptodate_indexer.index(page_doc)

In [9]:
vec = embed_model.get_embedding('điều trị hẹp hai lá')

In [11]:
a = uptodate_indexer.search('điều trị hẹp hai lá')

In [14]:
a[1][1]['doc'].get_doc_content()

'ĐIỀU TRỊ NHỖI MÁU NHĨ — Nhìn chung, phương pháp điều trị nội khoa đối với nhồi máu nhĩ ở những bệnh nhân hẹp van hai lá không khác biệt đáng kể so với các hướng dẫn điều trị chung. Tuy nhiên, do những tác động huyết động bất lợi quá mức do hẹp van hai lá gây ra, nên thường cần phải điều trị kịp thời. (Xem  "Nhồi máu nhĩ: Tổng quan và cách điều trị nhồi máu nhĩ khởi phát mới" và  "Kiểm soát nhịp thất ở những bệnh nhân nhồi máu nhĩ không suy tim: Liệu pháp dược lý" và \'Phòng ngừa huyết khối tắc mạch\' bên dưới.)\nỞ nhiều bệnh nhân hẹp van hai lá, sự khởi phát của nhồi máu nhĩ góp phần vào sự khởi phát các triệu chứng. Khi hẹp van hai lá có ý nghĩa huyết động (diện tích van ≤1,5 cm2), luồng máu qua van hai lá được duy trì bởi hai yếu tố: thời gian làm đầy tâm trương thích hợp và áp lực tĩnh mạch phổi cao hơn bình thường. Nhồi máu nhĩ ở những bệnh nhân hẹp van hai lá có thể không được dung nạp tốt vì hai lý do, với các hậu quả huyết động tùy thuộc vào mức độ hẹp:\n●Nếu nhồi máu nhĩ liên 

In [15]:
Base.close_instance(delete_model=True)